In [5]:
import pandas as pd
from statsmodels.formula.api import ols
import numpy as np

from linearmodels.panel import RandomEffects, PanelOLS

In [2]:
df = pd.read_stata("mymeap_ch2.dta")

In [12]:
df.dtypes

distid             float32
schid                int16
lunch              float32
enrol                int16
exppp                int16
year        datetime64[ns]
cpi                float32
rexppp             float32
lrexppp            float32
lenrol             float32
lenrolsq           float32
math4              float32
dtype: object

In [11]:
df

distid  schid      lunch  enrol  exppp       year    cpi       rexppp  \
0     34010.0      1  35.000000    248   2870 1994-01-01  1.482  3108.198486   
1     34010.0      1  36.590000    244   4176 1995-01-01  1.524  4397.953125   
2     34010.0      1  40.549999    247   4333 1996-01-01  1.569  4432.418945   
3     34010.0      1  49.490002    291   3666 1997-01-01  1.605  3666.000000   
4     34010.0      1  49.639999    274   3825 1998-01-01  1.630  3766.334473   
...       ...    ...        ...    ...    ...        ...    ...          ...   
7141  82010.0   7657  56.910000    208   4706 1998-01-01  1.630  4633.822266   
7142  47080.0   7720  10.900000    469   3619 1995-01-01  1.524  3811.348633   
7143  47080.0   7720  10.260000    497   3739 1996-01-01  1.569  3824.790039   
7144  47080.0   7720   9.430000    530   4021 1997-01-01  1.605  4021.000000   
7145  47080.0   7720   6.080000    510   4130 1998-01-01  1.630  4066.656494   

       lrexppp    lenrol   lenrolsq      math4  
0     8.041799  5.513429  30.397896  59.837444  
1     8.388894  5.497168  30.218857  68.370354  
2     8.396701  5.509388  30.353361  64.383919  
3     8.206857  5.673323  32.186596  69.927261  
4     8.233857  5.613128  31.507208  71.291817  
...        ...       ...        ...        ...  
7141  8.441137  5.337538  28.489315  69.067871  
7142  8.245738  6.150603  37.829914  79.299316  
7143  8.249259  6.208590  38.546589  83.668633  
7144  8.299286  6.272877  39.348988  78.292953  
7145  8.310576  6.234411  38.867878  82.490067  

[7146 rows x 12 columns]

In [6]:
df.describe()

distid        schid        lunch        enrol        exppp  \
count   7146.000000  7146.000000  7146.000000  7146.000000  7146.000000   
mean   52595.183594  2941.380493    36.732594   421.572488  3968.543661   
std    23716.892578  1993.758882    25.109406   163.979972   824.195808   
min     1010.000000     1.000000     0.000000    24.000000  1521.000000   
25%    33070.000000  1280.000000    15.887500   310.000000  3434.000000   
50%    54010.000000  2538.000000    32.195000   405.000000  3909.500000   
75%    75057.500000  4400.000000    53.230000   507.000000  4410.000000   
max    83010.000000  7720.000000   100.000000  1353.000000  8979.000000   

               cpi       rexppp      lrexppp       lenrol     lenrolsq  \
count  7146.000000  7146.000000  7146.000000  7146.000000  7146.000000   
mean      1.571562  4044.496338     8.286757     5.963970    35.746346   
std       0.050675   792.833435     0.190762     0.421001     4.868274   
min       1.482000  1539.027588     7.338906     3.178054    10.100026   
25%       1.524000  3515.525330     8.164944     5.736572    32.908260   
50%       1.569000  3968.190186     8.286065     6.003887    36.046661   
75%       1.605000  4459.884155     8.402879     6.228511    38.794346   
max       1.630000  9242.287109     9.131545     7.210080    51.985249   

             math4  
count  7146.000000  
mean     73.996811  
std       7.530081  
min      41.607044  
25%      69.159971  
50%      74.021896  
75%      78.953041  
max     100.000000

In [10]:
# Add a constant to the independent variable
X = sm.add_constant(df[["lrexppp", "lunch", "lenrol", "lenrolsq"]])
y = df["math4"]

# Fit the model
model = sm.OLS(y, X).fit()

# Print the summary of the model
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  math4   R-squared:                       0.566
Model:                            OLS   Adj. R-squared:                  0.566
Method:                 Least Squares   F-statistic:                     2328.
Date:                Mon, 06 Mar 2023   Prob (F-statistic):               0.00
Time:                        19:18:28   Log-Likelihood:                -21584.
No. Observations:                7146   AIC:                         4.318e+04
Df Residuals:                    7141   BIC:                         4.321e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -22.9911      5.977     -3.846      0.0

In [3]:
# Create dummy columns for the year
year_dummies = pd.get_dummies(df['year'].dt.year, prefix='year')

# Concatenate the dummy columns with the original dataframe
df = pd.concat([df, year_dummies], axis=1)

In [15]:
df.columns

Index(['distid', 'schid', 'lunch', 'enrol', 'exppp', 'year', 'cpi', 'rexppp',
       'lrexppp', 'lenrol', 'lenrolsq', 'math4', 'year_1994', 'year_1995',
       'year_1996', 'year_1997', 'year_1998'],
      dtype='object')

In [24]:
# Add a constant to the independent variable
X = sm.add_constant(df[
    [
        "lrexppp", "lunch", "lenrol", "lenrolsq", 'year_1995',
        'year_1996', 'year_1997', 'year_1998'
    ] ##taking year_1994 off because of multicollinearity
])
y = df["math4"]

# Fit the model
model = sm.OLS(y, X).fit(cov_type='HC3')

# Print the summary of the model
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  math4   R-squared:                       0.661
Model:                            OLS   Adj. R-squared:                  0.660
Method:                 Least Squares   F-statistic:                     1711.
Date:                Mon, 06 Mar 2023   Prob (F-statistic):               0.00
Time:                        20:02:41   Log-Likelihood:                -20704.
No. Observations:                7146   AIC:                         4.143e+04
Df Residuals:                    7137   BIC:                         4.149e+04
Df Model:                           8                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.9964      5.574      1.435      0.1

- Fixed effects model (still have to work on it: estimate white robust standard errors)

In [27]:
from linearmodels.panel import PanelOLS

In [28]:
df_panel = pd.read_stata("mymeap_ch2.dta")

In [ ]:
# Estimate the model with robust standard errors
model = PanelOLS.from_formula('y ~ x', data=df)
robust_model = model.fit(cov_type='robust')

# Print the summary of the model
print(robust_model.summary)

## Exercise questions - chapter 2

#### Question 1)

1a) A balanced panel mean that the number of time observations for each individual is the same

#### Question 2)

2a) Time fixed effect is specific to time and it effects all individuals at the same time
2b) We should take time variables into account on the moment of modeling, like dummy variables

#### Question 4)

4e) We need to run the joint significance test with the years dummy variables, if the p-value is smaller than the threshold, it means that time fixed effects is present 


In [41]:
from statsmodels.formula.api import ols

# Specify the full model
full_model = ols("math4 ~" + " + ".join([
        "lrexppp", "lunch", "lenrol", "lenrolsq", 'year_1995',
        'year_1996', 'year_1997', 'year_1998'
    ]), data=df)

# Specify the reduced model
reduced_model = ols("math4 ~" + " + ".join([
        "lrexppp", "lunch", "lenrol", "lenrolsq"
    ]), data=df)

# Fit both models
full_results = full_model.fit()
reduced_results = reduced_model.fit()

# Test joint significance of the years
f_test = full_results.compare_f_test(reduced_results)

# Print the F-statistic and p-value
print('F-statistic:', f_test[0])
print('p-value:', f_test[1])

F-statistic: 498.3659025858557
p-value: 0.0


#### Question 5)

5a) The goal is to find an estimator that is more efficient. We have to use FGLS

5b) We have to transform math4 with math4_it = math_it - lambda * math4_i(Bar)

$$ \lambda = 1 - \sqrt{\frac{1}{1 + (\frac{\sigma_c^2}{\sigma^2_u})T}}

Breusch and pagan lagrangian multiplier test for random effects

#### Question 6)



In [23]:
import statsmodels.api as sm
fixed_effects_df = df.set_index(['schid', 'year'])

year_dummy_columns = list(year_dummies.columns)[1:] # year_1994 is removed to avoid multicollinearity
exog_vars = ["lrexppp", "lunch", "lenrol", "lenrolsq"]
exog = sm.add_constant(fixed_effects_df[exog_vars+year_dummy_columns])

mod = PanelOLS(fixed_effects_df.math4, exog)
fe_res = mod.fit()

print(fe_res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  math4   R-squared:                        0.6608
Estimator:                   PanelOLS   R-squared (Between):              0.8364
No. Observations:                7146   R-squared (Within):               0.3326
Date:                Mon, Mar 13 2023   R-squared (Overall):              0.6608
Time:                        19:04:00   Log-likelihood                 -2.07e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      1737.6
Entities:                        1683   P-value                           0.0000
Avg Obs:                       4.2460   Distribution:                  F(8,7137)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             1737.6
                            

/Users/gabrieldiasmp/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


## My notes about Panel Data methods

#### Fixed Effects

In fixed effects, we have to take into account the effects of group characteristics that are fixed in time. We can test the significance of group differences using an F-Test, comparing to the Pooled OLS:

<img src='images/testing_group_effects.png' width=50% height=50%>

##### Random Effects